In [ ]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder().appName("sesion_1").master("local[*]").getOrCreate()
val sc = spark.sparkContext

In [ ]:
sc.uiWebUrl

In [ ]:
val contractsDf  = spark.read.option("header","true")
        .option("delimiter",",")
        .option("inferSchema","true")
        .csv("../../resources/data/csv/contracts.csv")

In [ ]:
contractsDf.show(2)

In [ ]:
contractsDf.printSchema()

#### - Clase Column
#### - Package Types
#### - Object functions
#### - Clase Dataframe

### Transformaciones

In [ ]:
import org.apache.spark.sql.{functions => f}
import org.apache.spark.sql.{types => t}

val resultedDf = contractsDf
    .withColumnRenamed("fec_alta", "fec_alta_ini")
    .withColumn("actual_date", f.current_date())
    .withColumn("randm_num", f.round(f.rand(0) * f.lit(10)).cast(t.IntegerType))
    .withColumn("fec_alta_fin", f.date_add(f.col("fec_alta_ini"), f.col("randm_num")))
    .withColumn("diff", f.datediff(f.col("fec_alta_fin"), f.col("fec_alta_ini")))
    .drop("randm_num")

### Acciones

In [ ]:
resultedDf.show(2, false)

In [ ]:
resultedDf.count()

In [ ]:
resultedDf.collect() // WARNING

In [ ]:
resultedDf.isEmpty

In [ ]:
//first() -> primer elemento del df -> head()
//head(n) -> primer N-elementos del df -> take(n)
//take -> primer N-elementos del df -> limit().collect()

resultedDf.take(4)

In [ ]:
resultedDf.tail(4)

In [ ]:
resultedDf.describe("cod_iuc","cod_titular").show()

In [ ]:
resultedDf.summary("count", "stddev").show()

### Functions

In [ ]:
resultedDf.columns

In [ ]:
import org.apache.spark.sql.types._
import org.apache.spark.sql.Row

val schema_nest = StructType( Array(
    StructField("alumno",StructType( Array(
      StructField("nombre", StringType,true),
      StructField("apellido_p", StringType,true),
      StructField("apellido_m", StringType,true)
    ))),
    StructField("calificaciones", StructType( Array(
        StructField("matematicas",StructType( Array(
            StructField("periodo_1", IntegerType,true),
            StructField("periodo_2", IntegerType,true),
            StructField("periodo_3", IntegerType,true)
        ))),
        StructField("espanol",StructType( Array(
            StructField("periodo_1", IntegerType,true),
            StructField("periodo_2", IntegerType,true),
            StructField("periodo_3", IntegerType,true)
        ))),
    )))
  ))

//Create DataFrame
val data = List(
    Row(Row("Pedro","Jimenez","Perez"), Row(Row(10, 9, 8), Row(8, 9, 10))),
    Row(Row("Juan","Hernandez","Perez"), Row(Row(10, 10, 8), Row(9, 9, 10)))
    )
val df = spark.createDataFrame(
    spark.sparkContext.parallelize(data),schema_nest)
df.show(2, false)
df.printSchema()

In [ ]:
df.printSchema(3)

In [ ]:
resultedDf.schema

In [ ]:
resultedDf.dtypes

In [ ]:
resultedDf.rdd.collect().foreach(println)

In [ ]:
resultedDf.explain("formatted")